In [4]:
using InteractiveUtils, Pkg, Revise
using AdvancedHMC; Pkg.activate(pkgdir(AdvancedHMC) * "/research/tests"); Pkg.instantiate()
versioninfo(); Pkg.status()

[ Info: Precompiling AdvancedHMC [0bf59076-c3b1-5ca4-86bd-e02cd72cde3d]
ERROR: LoadError: ArgumentError: Package AdvancedHMC does not have SimpleUnPack in its dependencies:
- You may have a partially installed environment. Try `Pkg.instantiate()`
  to ensure all packages in the environment are installed.
- Or, if you have AdvancedHMC checked out for development and have
  added SimpleUnPack as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with AdvancedHMC
Stacktrace:
 [1] macro expansion
   @ ./loading.jl:1597 [inlined]
 [2] macro expansion
   @ ./lock.jl:267 [inlined]
 [3] require(into::Module, mod::Symbol)
   @ Base ./loading.jl:1574
 [4] include
   @ ./Base.jl:457 [inlined]
 [5] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt128}}, source::Nothing)
  

LoadError: Failed to precompile AdvancedHMC [0bf59076-c3b1-5ca4-86bd-e02cd72cde3d] to "/Users/hg344/.julia/compiled/v1.9/AdvancedHMC/jl_xsuus6".

In [ ]:
using PyPlot: PyPlot; const plt = PyPlot
using Statistics, AdvancedHMC
includet("../src/riemannian_hmc.jl")
includet("../src/riemannian_hmc_utility.jl")

Baseline: HMC with dense but identity Euclidean metric

In [ ]:
hps = (; target=:gaussian, n_samples=2_000, metric=:dense_euclidean, λ=1e-2, α=20.0, integrator=:lf, ϵ=0.1, n=6, L=8)
retval = sample_target((; hps...))

@info "Samples" mean(retval.samples) var(retval.samples)

Check if generalized leapfrog works fine

In [ ]:
retval = sample_target((; hps..., integrator=:glf))

@info "Samples" mean(retval.samples) var(retval.samples)

Check if RHMC (with Fisher metric) works (on Gaussian)

In [ ]:
@time retval = sample_target((; hps..., metric=:dense_riemannian, integrator=:glf))

@info "Samples" mean(retval.samples) var(retval.samples)

Check if RHMC (with Fisher metric) works for Neal's funnel

In [ ]:
using Logging: NullLogger, with_logger

@time retval = with_logger(NullLogger()) do 
    sample_target((; hps..., target=:funnel, metric=:dense_riemannian, integrator=:glf))
end

@info "Average acceptance ratio" mean(map(s -> s.is_accept, retval.stats))

let (fig, ax) = plt.subplots()
    plt.close(fig)
    
    ax.scatter(map(s -> s[1], retval.samples), map(s -> s[2], retval.samples))
    
    fig
end

Check if RHMC with SoftAbs works for Gaussian

In [ ]:
@time retval = sample_target((; hps..., metric=:dense_riemannian_softabs, integrator=:glf))

@info "Samples" mean(retval.samples) var(retval.samples)

Check if RHMC with SoftAbs works for Neal's funnel

In [ ]:
@time retval = with_logger(NullLogger()) do 
    sample_target((; hps..., target=:funnel, metric=:dense_riemannian_softabs, integrator=:glf, ϵ=0.2))
end

@info "Average acceptance ratio" mean(map(s -> s.is_accept, retval.stats))

let (fig, ax) = plt.subplots()
    plt.close(fig)
    
    ax.scatter(map(s -> s[1], retval.samples), map(s -> s[2], retval.samples))
    
    fig
end

speed through optimization
- unoptimized: 200s
- +cache 1st loop: 82s
- +cache 2nd loop: 77s
- -copy in ad: 75s
- -rev +fwd: 11s

Check if the Hessian is (semi)-PD for Neal's funnel

In [ ]:
using PyCall: pyimport
using LinearAlgebra: issuccess, cond
using Printf: @sprintf
const axes_grid1 = pyimport("mpl_toolkits.axes_grid1")

isposdef_pivot(A) = issuccess(cholesky(A, Val(true), check=false))

function make_Z(Gfunc, f, λ=0.0)
    is = -6:0.1:+2
    js = -4:0.1:+4
    X = Matrix{Float64}(undef, length(is), length(js))
    Y = Matrix{Float64}(undef, length(is), length(js))
    logdensity = Matrix{Float64}(undef, length(is), length(js))
    Z = Matrix(undef, length(is), length(js))
    for (i, iv) in enumerate(is), (j, jv) in enumerate(js)
        v, _, G = Gfunc([iv, jv])
        X[i,j] = iv
        Y[i,j] = jv
        logdensity[i,j] = exp(-v)
        # PD[i,j] = isposdef(H + λ * I)
        Z[i,j] = f(G + λ * I)
    end
    return logdensity, X, Y, Z
end

function plot_contour!(ax, X, Y, Z, label=nothing; kwargs...)
    contour = ax.contour(X, Y, Z, alpha=0.7; kwargs...)
    divider = axes_grid1.make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(contour, cax=cax, ax=ax, orientation="vertical")
    isnothing(label) || cbar.set_label(label)
    ax.axis("equal")
end

# Check which region gives non-positive definite for Neal's funnel
let target = Funnel(), 
    rng = MersenneTwister(1110), 
    
    D = dim(target)
    initial_θ = rand(rng, D)
    Vfunc = x -> -logpdf(target, x)
    Gfunc = VecTargets.gen_hess(Vfunc, initial_θ)
    
    logdensity, X, Y, PD = make_Z(Gfunc, isposdef_pivot)
    _, _, _, C = make_Z(Gfunc, cond)
    
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4; figsize=(4 * 4, 4), tight_layout=true); plt.close(fig)
    
    plot_contour!(ax1, X, Y, logdensity, "density")
    
    ax2.imshow(PD', cmap=plt.cm.gray; interpolation="nearest", extent=[extrema(X)..., extrema(Y)...]) 
    ax2.set_title("is PD? black=0, white=1")
    
    Cvec = vec(C)
    Cv95 = quantile(Cvec, 0.95) # ignoring outliers
    
    plot_contour!(ax3, X, Y, C; levels=[1:9..., 10:10:50...])
    ax2.set_title("cond. number")
    
    ax4.hist(Cvec[Cvec .< Cv95]; bins=30, density=true)
    minstr = @sprintf("%.3f", minimum(Cvec))
    maxstr = @sprintf("%.3f", maximum(Cvec))
    ax4.set_title("min=$minstr, max=$maxstr")
    
    fig
end